In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [3]:
path = "../datasets/Shared_Dataset/"
files_list = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
        # checking if it is a file
    if os.path.isfile(f):
        files_list.append(f)
print("Number of files: {}".format(len(files_list)))

Number of files: 14


## Merge Dataset

In [6]:
i = 0
df = pd.DataFrame()
drop_list = ['Data','Hora', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
while i < len(files_list):
    print(files_list[i])
    data = pd.read_excel(files_list[i], sheet_name=['Total_Consumers'])
    df_tmp = data['Total_Consumers']
    df_tmp['Data'] = df_tmp['Data'].apply(lambda x: str(x).replace('/', '-'))
    df_tmp['Time'] = df_tmp['Data'] + ' ' + df_tmp['Hora']
    df_tmp['Time'] = pd.to_datetime(df_tmp['Time'], infer_datetime_format=True)
    tmp_filter = df_tmp.filter(drop_list)
    df_tmp.drop(tmp_filter, axis=1, inplace=True)
    df_tmp.dropna(subset=['Time'], inplace=True)
    df_tmp = df_tmp.iloc[:,[1,0]]
    df_tmp = df_tmp.drop_duplicates(subset=['Time'])
    print(df_tmp.shape)
    print("Time range" + ": " + str(min(df_tmp['Time'])) + " - " + str(max(df_tmp['Time'])))
    if len(df) == 0:
        df = df_tmp
    else:
        df = df.merge(df_tmp, on='Time', how='outer')
    #name = files_list[i].split("/")[3]
    #name = name[0:len(name)-5]
    #name = name[0:2] + name[len(name)-4:]
    df.rename(columns={'Consumo Registado': i}, inplace=True)
    print(df.shape)
    i += 1
df

../datasets/Shared_Dataset/.xlsx
(14689, 2)
Time range: 2022-09-15 01:00:00 - 2023-02-15 01:00:00
(14689, 2)
../datasets/Shared_Dataset/PT0002000080864815TS.xlsx
(23323, 2)
Time range: 2022-08-01 01:15:00 - 2023-04-01 01:45:00
(23323, 3)
../datasets/Shared_Dataset/PT0002000077861448JB.xlsx
(39545, 2)
Time range: 2022-01-01 00:00:00 - 2023-02-17 00:00:00
(43672, 4)
../datasets/Shared_Dataset/PT0002000052213942JN.xlsx
(29955, 2)
Time range: 2022-01-21 00:00:00 - 2022-11-29 01:30:00
(43676, 5)
../datasets/Shared_Dataset/PT0002000009906944AL.xlsx
(39260, 2)
Time range: 2021-10-19 00:15:00 - 2022-12-02 00:00:00
(50779, 6)
../datasets/Shared_Dataset/PT0002000009686817RW.xlsx
(81584, 2)
Time range: 2020-11-01 00:00:00 - 2023-02-28 23:45:00
(84564, 7)
../datasets/Shared_Dataset/PT0002000071112543QL.xlsx
(69208, 2)
Time range: 2021-01-01 00:15:00 - 2022-12-23 00:00:00
(84568, 8)
../datasets/Shared_Dataset/PT0002000110339041CV.xlsx
(75448, 2)
Time range: 2020-10-06 00:15:00 - 2022-12-01 00:00:00

,Time,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2022-09-15 01:00:00,0.128,0.644,0.060,0.244,0.412,0.052,0.100,0.144,NaN,0.440,0.288,0.252,0.540,0.164
1,2022-09-15 01:15:00,0.088,1.068,0.048,0.240,0.412,0.028,0.112,0.164,NaN,0.464,0.224,0.232,0.444,0.188
2,2022-09-15 01:30:00,0.084,0.768,0.144,0.112,0.380,0.028,0.100,0.116,NaN,0.332,0.220,0.280,0.244,0.124
3,2022-09-15 01:45:00,0.080,0.684,0.028,0.236,0.528,0.040,0.096,0.132,NaN,0.152,0.232,0.224,0.192,0.124
4,2022-09-15 02:00:00,0.076,0.644,0.076,0.152,0.476,0.084,0.092,0.124,NaN,0.144,0.240,0.276,0.196,0.168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87058,2020-10-31 22:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004,NaN,0.648,NaN,NaN,NaN,NaN
87059,2020-10-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,0.544,NaN,NaN,NaN,NaN
87060,2020-10-31 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004,NaN,0.512,NaN,NaN,NaN,NaN
87061,2020-10-31 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004,NaN,0.520,NaN,NaN,NaN,NaN


In [4]:
df_sync = df.dropna()
df_sync.reset_index(inplace=True, drop=True)
print("Time range" + ": " + str(min(df_sync['Time'])) + " - " + str(max(df_sync['Time'])))
df_sync

Time range: 2021-11-01 00:15:00 - 2022-11-01 00:00:00


,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35027,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35028,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35029,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35030,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


In [5]:
dplctd = df_sync.duplicated(subset=['Time']).sum()
dplctd

0

In [6]:
dplctd = df.duplicated(subset=['Time']).sum()
dplctd

0

In [7]:
df.sort_values(by=['Time'], inplace=True)
df

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2020-10-06 00:15:00,0.076,NaN,NaN,NaN,NaN,NaN,0.320
1,2020-10-06 00:30:00,0.072,NaN,NaN,NaN,NaN,NaN,0.308
2,2020-10-06 00:45:00,0.072,NaN,NaN,NaN,NaN,NaN,0.292
3,2020-10-06 01:00:00,0.068,NaN,NaN,NaN,NaN,NaN,0.280
4,2020-10-06 01:15:00,0.032,NaN,NaN,NaN,NaN,NaN,0.340
...,...,...,...,...,...,...,...,...
84082,2023-02-28 22:45:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84083,2023-02-28 23:00:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84084,2023-02-28 23:15:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN
84085,2023-02-28 23:30:00,NaN,NaN,NaN,NaN,NaN,0.064,NaN


In [7]:
for feature in df.columns:
    print(feature + ": " + str(min(df[feature])) + " - " + str(max(df[feature])))

Time: 2020-10-06 00:15:00 - 2023-04-01 01:45:00


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [9]:
df_test = df_sync.copy()
df_test = df_test.set_index("Time")
df_test

,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
Time,,,,,,,
2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...
2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356


### Missing 8 values from 1:00 am to 3:00 am from 27/03/2022 in all sheets due to time change

In [10]:
df1 = df_test.asfreq('15T')
df1

,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
Time,,,,,,,
2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...
2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356


In [11]:
inds = pd.isnull(df1.values).any(1).nonzero()[0]
inds

array([14019, 14020, 14021, 14022, 14023, 14024, 14025, 14026])

In [12]:
a = df1.iloc[inds[:]]
a

,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
Time,,,,,,,
2022-03-27 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 01:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 01:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-03-27 02:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df1['Time'] = df1.index
cols = df1.columns.drop('Time')
cols = cols.insert(0, 'Time')
df1.reset_index(drop=True, inplace=True)
df1 = df1.reindex(columns=cols)
df1

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35035,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35036,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35037,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35038,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


In [8]:
df.to_csv("data/shared_records.csv")

In [15]:
df_sync.to_csv("shared_1year.csv", index=False)

## Merge with weather data

In [ ]:
df_weather = pd.read_csv("../")